# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,uturoa,-16.7333,-151.4333,75.87,74,96,17.00,PF,1722738831
1,1,kailua-kona,19.6406,-155.9956,84.16,84,40,10.36,US,1722739023
2,2,jamestown,42.0970,-79.2353,71.19,97,37,1.01,US,1722738748
3,3,ujae,8.9322,165.7642,83.46,75,22,8.43,MH,1722739026
4,4,edinburgh of the seven seas,-37.0676,-12.3116,54.79,84,100,35.43,SH,1722739028


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    color="City",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600
)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities = city_data_df.loc[(city_data_df["Max Temp"] > 40) & (city_data_df["Max Temp"] < 70) & (city_data_df['Cloudiness'] > 50)]

# Drop any rows with null values
my_cities = my_cities.dropna()

# Display sample data
my_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,edinburgh of the seven seas,-37.0676,-12.3116,54.79,84,100,35.43,SH,1722739028
5,5,papatowai,-46.5619,169.4708,50.31,59,94,5.39,NZ,1722739029
12,12,isafjordur,66.0755,-23.1240,45.07,95,100,10.45,IS,1722739040
16,16,tommot,58.9564,126.2925,67.86,65,100,7.09,RU,1722739046
18,18,puerto ayora,-0.7393,-90.3518,67.55,93,77,7.67,EC,1722739049
...,...,...,...,...,...,...,...,...,...,...
561,561,grand falls-windsor,48.9332,-55.6649,69.51,72,100,8.39,CA,1722739910
562,562,borskoye,53.0333,51.7167,65.46,67,54,6.53,RU,1722739911
565,565,atbasar,51.8000,68.3333,63.39,72,100,14.97,KZ,1722739916
568,568,flekkefjord,58.2971,6.6607,61.95,99,99,0.74,NO,1722739920


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,84,
5,papatowai,NZ,-46.5619,169.4708,59,
12,isafjordur,IS,66.0755,-23.1240,95,
16,tommot,RU,58.9564,126.2925,65,
18,puerto ayora,EC,-0.7393,-90.3518,93,
...,...,...,...,...,...,...
561,grand falls-windsor,CA,48.9332,-55.6649,72,
562,borskoye,RU,53.0333,51.7167,67,
565,atbasar,KZ,51.8000,68.3333,72,
568,flekkefjord,NO,58.2971,6.6607,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
api_key = geoapify_key

params = {}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["categories"] = "accommodation.hotel"
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["limit"] = limit
    params["apiKey"] = api_key

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    hotel = requests.get(base_url, params=params)
    hotel.raise_for_status()

    # Convert the API response to JSON format
    hotel = hotel.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
tommot - nearest hotel: Томмот
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
lakota - nearest hotel: Hôtel Gnoupalé
tazovsky - nearest hotel: Тазовчанка
bredasdorp - nearest hotel: Victoria Hotel
keflavik - nearest hotel: Núpan Deluxe
waitangi - nearest hotel: Hotel Chathams
iqaluit - nearest hotel: Frobisher Inn
mampong - nearest hotel: E Hotel
kerikeri - nearest hotel: Avalon Resort
weisswasser - nearest hotel: Hotel Kristall
longyearbyen - nearest hotel: The Vault
bardai - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
vuktyl - nearest hotel: Вуктыл
yuzhno-kurilsk - nearest hotel: Айсберг
qaqortoq - nearest hotel: Hotel Qaqortoq
lebu - nearest hotel: No hotel found
salym - nearest hotel: Каравани
aykhal - nearest hotel: Уют
port alfred - nearest hotel: No hotel found
college - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,84,No hotel found
5,papatowai,NZ,-46.5619,169.4708,59,No hotel found
12,isafjordur,IS,66.0755,-23.1240,95,Hótel Horn
16,tommot,RU,58.9564,126.2925,65,Томмот
18,puerto ayora,EC,-0.7393,-90.3518,93,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...
561,grand falls-windsor,CA,48.9332,-55.6649,72,Hotel Robin Hood
562,borskoye,RU,53.0333,51.7167,67,Элегия
565,atbasar,KZ,51.8000,68.3333,72,Дана
568,flekkefjord,NO,58.2971,6.6607,99,Grand hotell Flekkefjord


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    color="City",
    hover_cols = ["Hotel Name", "Country"],
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)